In [1]:
%config IPCompleter.greedy=True
%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
from IPython.display import display, clear_output
import ipywidgets as widgets
import numpy as np

import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import psutil as ps

import time
import threading

InteractiveShell.ast_node_interactivity = 'all'

In [2]:
class JTop(object):
    def __init__(self):
        self.thread = None
    
    def __monitor(self, out):
        def col(x):
            if 0 < x < 75:
                return '#142940'
            elif 75 <= x < 95:
                return '#28537F'
            else:
                return '#3D7CBF'

        def remove_spines_and_ticks(ax):
            ax.set_xticks([])
            ax.tick_params(axis='both', which='both', length=0)

            ax.spines['top'].set_visible(False)
            ax.spines['bottom'].set_visible(False)
            ax.spines['left'].set_visible(False)
        
        ln_cpu_freq, len_cpu_perc = [], []
        while True:
            for i in range(20):
                ln_cpu_perc.append(cpu_percent(percpu=True))
                ln_cpu_perc = ln_cpu_perc[1:]
                time.sleep(.05)
            cpu_perc = np.array(ln_cpu_perc).T
            cpu_perc = np.mean(cpu_perc, axis=0)
            
            with out:            
                rows = int(np.ceil(len(cpu_perc)/2)) + 1
                cols = 2

                fig = plt.figure(figsize=(15, 2))
                fig.set_tight_layout(True)
                gs = gridspec.GridSpec(rows, 2)

                i, j = 0, 0
                for core, perc in enumerate(cpu_perc, 1):
                    ax = plt.subplot(gs[i, j])

                    i = (i + 1)%(rows - 1)
                    if i == 0:
                        j += 1                

                    ax.barh(
                        y=core,
                        width=perc,
                        color=col(perc),
                        align='center'
                    )
                    ax.set_ylim([core-.01, core+.01])
                    ax.set_xlim([0, 110])

                    ax.set_yticklabels([core], fontsize=16)
                    ax.set_yticks([core])

                    remove_spines_and_ticks(ax)
                    ax.annotate(f'{perc}%', xy=(109, core), fontsize=16, horizontalalignment='right')

                mem = ps.virtual_memory()

                ax = plt.subplot(gs[rows-1, :cols])
                ax.barh(
                    y=0,
                    width=mem.percent,
                    color=col(mem.percent),
                    align='center'
                )

                ax.set_yticks([0])
                ax.set_yticklabels(['Mem'], fontsize=16)
                ax.set_xlim([0, 110])

                remove_spines_and_ticks(ax)

                ax.annotate(f'{mem.percent}%', xy=(109, 0), fontsize=16, horizontalalignment='right')
                ax.tick_params(axis='both', which='both', length=0)

                out.append_stdout(print(''))
                out.clear_output(wait=True)

    def start(self):    
        out = widgets.Output(layout={
            'height':'150px',
            'width':'100%'
        })

        display(out)

        self.thread = threading.Thread(target=self.__monitor, args=(out,))
        self.thread.start()

        return self
    
    def stop(self):    
        self.thread.join()

In [3]:
jtop = JTop().start()

Output(layout=Layout(height='150px', width='100%'))

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/home/luke/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/luke/anaconda3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-794c1675ce42>", line 25, in __monitor
    ln_cpu_perc.append(cpu_percent(percpu=True))
UnboundLocalError: local variable 'ln_cpu_perc' referenced before assignment

